# Import Packages

In [2]:
using CSV
using Plots
using Measures
using Flux
using DifferentialEquations
using DiffEqFlux
using LaTeXStrings

# Import Dataset

In [3]:
confirmed_fn = CSV.read("time_series_covid19_confirmed_global.csv")
dead_fn = CSV.read("time_series_covid19_deaths_global.csv")
recovered_fn = CSV.read("time_series_covid19_recovered_global.csv")
Confirmed = Array(confirmed_fn[226,51:129])
Dead = Array(dead_fn[226,51:129])
Recovered = Array(recovered_fn[226,51:129])
Infected = Confirmed .- Dead .- Recovered
Time = collect(1:1:(129+1)-51)

79-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 68
 69
 70
 71
 72
 73
 74
 75
 76
 77
 78
 79

# Taking Gradient Using Flux
## Documentation: https://fluxml.ai/Flux.jl/stable/models/basics/

In [4]:
# ann means Artificial Neural Networks
# ReLU means Rectified Linear Unit, which is a layer activation function here
# Chain multiple layers / functions together, so that they are called in sequence on a given input.

# Here is a n = 2-layer densely connected neural network with 10 units in the hidden layer (See Figure 11)
ann = Chain(Dense(4,10,relu), Dense(10,1))
# Flatten a model's parameters into a single weight vector.
# The second return value re allows you to reconstruct the original network after making modifications to the
# weight vector.
p1,re = Flux.destructure(ann)
# beta and gamma
p2 = Float64[0.5, 0.03]
# The final model was described by 63 tunable parameters which are in p3, p1 has 61 plus p2 has 2.
p3 = [p1; p2]
# make p3 into params format
ps = Flux.params(p3)

Params([[-0.15756171941757202, -0.07919720560312271, -0.2679900825023651, -0.3180989921092987, 0.19072562456130981, 0.4660479426383972, 0.5790422558784485, -0.15663959085941315, 0.16085660457611084, -0.5511414408683777  …  0.7205072045326233, -0.6869215369224548, -0.5823696851730347, 0.35903361439704895, 0.5721712112426758, 0.28743284940719604, -0.09562107175588608, 0.0, 0.5, 0.03]])

In [5]:
# u = [S,I,R,T]
# du = [dS,dI,dR,dT]
# u0 = [S0,I0,R0,T0]
# ODEProblem Dodumentation: https://docs.sciml.ai/stable/types/ode_types/

#ODE Function
function QSIR(du, u, p, t)
    β = abs(p[62])
    γ = abs(p[63])
    du[1]=  - β*u[1]*(u[2])/u0[1]
    du[2] = β*u[1]*(u[2])/u0[1] - γ*u[2] - abs(re(p[1:61])(u)[1])*u[2]/u0[1]
    du[3] = γ*u[2]
    du[4] =  abs(re(p[1:61])(u)[1]*u[2]/u0[1])
end

QSIR (generic function with 1 method)

In [6]:
# Initial Condition
u0 = Float64[328200000.0, 491, 28, 10]
# Time Space
tspan = (0, 79.0)
datasize = (129 + 1) - 51

79

In [7]:
# ODEProblem Dodumentation: https://docs.sciml.ai/stable/types/ode_types/
# Local Sensitivity Analysis Documentation: https://docs.sciml.ai/stable/analysis/sensitivity/#

prob = ODEProblem(QSIR, u0, tspan, p3)
t = range(tspan[1],tspan[2],length=datasize)

sol = Array(concrete_solve(prob, Rosenbrock23(autodiff = false),u0, p3, saveat=t))

4×79 Array{Float64,2}:
   3.282e8    3.282e8    3.28199e8  …  8.90292e7  8.87339e7  8.8459e7
 491.0      680.172    942.325         2.22615e6  2.07758e6  1.94019e6
  28.0       45.6409    70.0876        2.56892e7  2.57546e7  2.58158e7
  10.0       97.2024   218.047         2.11256e8  2.11634e8  2.11986e8

In [8]:
function predict_adjoint() # Our 1-layer neural network
  Array(concrete_solve(prob,Rosenbrock23(autodiff = false),u0,p3,saveat=t))
end

predict_adjoint (generic function with 1 method)

In [9]:
I = Infected[1, :];
R = Recovered[1,:] + Dead[1,:];

In [10]:
function loss_adjoint()
 prediction = predict_adjoint()
 loss = sum(abs2, log.(abs.(Infected)) .- log.(abs.(prediction[2, :]))) + sum(abs2, log.(abs.(Recovered + Dead) .+ 1) .- log.(abs.(prediction[3, :] .+ 1)))
end

loss_adjoint (generic function with 1 method)

In [11]:
Loss = []
P1 = []
P2 = []
P3 = []

0-element Array{Any,1}

In [12]:
anim = Animation()

# Times of iterations
datan = Iterators.repeated((), 7000)
opt = ADAM(0.1)

# callback function to observe training
cb = function()
  display(loss_adjoint())
  #scatter(Time, Infected, xaxis = "Time(Days)", yaxis = "US - Number", label = "Data: Infected", legend = :topleft, framestyle = :box, left_margin = 5mm)
  prediction = solve(remake(prob,p=p3),Rosenbrock23(autodiff = false),saveat=Time)
  #display(scatter!(t, prediction[2, :], label = "NN - Infected"))
  #scatter!(Time, Recovered + Dead, xaxis = "Time(Days)", yaxis = "US - Number", label = "Data: Recovered + Dead", legend = :topleft, framestyle = :box, left_margin = 5mm)
  #display(scatter!(t, prediction[3, :], label = "NN - Recovered + Dead"))
  global Loss = append!(Loss, loss_adjoint())
  global P1 = append!(P1, p3[62])
  global P2 = append!(P2, p3[63])
  global P3 = append!(P3, p3)
  frame(anim)
end

#3 (generic function with 1 method)

In [13]:
# Display the ODE with the current parameter values
cb()

1478.7689443432537

ErrorException: No current plot/subplot

In [14]:
# Stop Iterations when loss function starts to stagnate and derivative of solution matches derivative of data
# at end point
Flux.train!(loss_adjoint, ps, datan, opt, cb = cb)


16006.178241716181

ErrorException: No current plot/subplot

In [ ]:
# Make a gif of training process
gif(anim,"Dead_US.gif", fps=15)

In [ ]:
L = findmin(Loss)

# Since we have 63 parameters from the neural network setting, the following steps is to get the 63 parameters
# in P3 as our final parameters
idx = L[2]
idx1 = (idx-1)*63 +1
idx2 = idx*63
p3 = P3[idx1: idx2]

In [ ]:
# Use the parameter we got in prediction, these prediction should fit the training set and potentially fit the
# prediction set
prediction = Array(concrete_solve(prob,Rosenbrock23(autodiff = false),u0,p3,saveat=t))

In [ ]:
S_NN_all_loss = prediction[1, :]
I_NN_all_loss = prediction[2, :]
R_NN_all_loss = prediction[3, :]
T_NN_all_loss = prediction[4, :]

In [ ]:
# Generlize a zero vector to represent quarentine strength timewise
Q_parameter = zeros(Float64, length(S_NN_all_loss), 1)

In [ ]:
# remake under the final parameters
for i = 1:length(S_NN_all_loss)
  Q_parameter[i] = abs(re(p3[1:61])([S_NN_all_loss[i],I_NN_all_loss[i], R_NN_all_loss[i], T_NN_all_loss[i]])[1])
end

In [ ]:
# Graph: Infected and recovered count

scatter(Time, Infected, xaxis = "Days post 500 infected", yaxis = "US: Number of cases", label = "Data: Infected", legend = :topleft, framestyle = :box, left_margin = 5mm, color = :red)
plot!(t, prediction[2, :], xaxis = "Days post 500 infected", yaxis = "US: Number of cases", label = "Prediction: Infected", legend = :topright, framestyle = :box, left_margin = 5mm, bottom_margin = 5mm, top_margin = 5mm,  grid = :off, color = :red, linewidth  = 3, ylims = (0, 1300000), foreground_color_legend = nothing, background_color_legend = nothing, yguidefontsize = 14, xguidefontsize = 14,  xtickfont = font(12, "TimesNewRoman"), ytickfont = font(12, "TimesNewRoman"), legendfontsize = 12)
scatter!(Time, Recovered + Dead, xaxis = "Days post 500 infected", yaxis = "US: Number of cases", label = "Data: Recovered + Dead", legend = :topleft, framestyle = :box, left_margin = 5mm, color = :blue)
plot!(t, prediction[3, :], xaxis = "Days post 500 infected", yaxis = "US: Number of cases", label = "Prediction: Recovered + Dead ", legend = :topleft, framestyle = :box, left_margin = 5mm, bottom_margin =5mm, top_margin = 5mm, grid = :off, color = :blue, linewidth  = 3, ylims = (0, 100000), foreground_color_legend = nothing, background_color_legend = nothing,  yguidefontsize = 14, xguidefontsize = 14,  xtickfont = font(12, "TimesNewRoman"), ytickfont = font(12, "TimesNewRoman"), legendfontsize = 12)
# if want to print Susceptible subclass, but the number of this class is way too large compared to the other two subclasses, which
# might not present a good graphing in a proper scale 
plot!(t, prediction[1, :], xaxis = "Days post 500 infected", yaxis = "US: Number of cases", label = "Prediction: Susceptible", legend = :topright, framestyle = :box, left_margin = 5mm, bottom_margin = 5mm, top_margin = 5mm,  grid = :off, color = :red, linewidth  = 3, ylims = (0, 1300000), foreground_color_legend = nothing, background_color_legend = nothing, yguidefontsize = 14, xguidefontsize = 14,  xtickfont = font(12, "TimesNewRoman"), ytickfont = font(12, "TimesNewRoman"), legendfontsize = 12)
savefig("US_1dn.pdf")

In [ ]:
# Graph: Quarantine strength
scatter(t,Q_parameter/u0[1], ylims = (0.0, 0.5), xlabel = "Days post 500 infected", ylabel = "Q(t)", label = "Quarantine Strength",color = :black, framestyle = :box, grid =:off, legend = :topleft, left_margin = 5mm, bottom_margin = 5mm, foreground_color_legend = nothing, background_color_legend = nothing,  yguidefontsize = 14, xguidefontsize = 14,  xtickfont = font(12, "TimesNewRoman"), ytickfont = font(12, "TimesNewRoman"), legendfontsize = 12)
savefig("US_2dn.pdf")

In [ ]:
# Graph: Reproduction number
scatter(t, abs(p3[62]) ./ (abs(p3[63]) .+ Q_parameter/u0[1]), ylims= (0.9, 2.5),  xlabel = "Days post 500 infected", ylabel = L"R_{t}", label = "Effective reproduction number", legend = :topright, color = :black, framestyle = :box, grid =:off, foreground_color_legend = nothing, background_color_legend = nothing, yguidefontsize = 14, xguidefontsize = 14,  xtickfont = font(12, "TimesNewRoman"), ytickfont = font(12, "TimesNewRoman"), legendfontsize = 12, left_margin = 5mm, bottom_margin= 5mm)
f(x) = 1
plot!(f, color = :blue, linewidth = 3, label = L"R_{t} = 1")
savefig("US_3dn.pdf")